# TAYSIR - Transformer for Track 1 (Binary Classification)
This notebook gives some clues about our transformers. Unfortunately, we do not have a baseline for transformers trained for classification since there does not exist, as far as we know, an algorithm to extract finite state automata on that task.

The only Transformer of Track 1 is Dataset 1.7

## Loading the model
Our Transformers come from the Hugging Face version of DistilBert. All information can be found here: https://huggingface.co/docs/transformers/model_doc/distilbert

First step is thus to make sure you have this installed:

In [ ]:
!pip3 install transformers

In [ ]:
TRACK = 1  #always for this track
DATASET = 7

In [ ]:
import mlflow, torch, transformers

model_name = f"models/{TRACK}.{DATASET}.taysir.model"

model = mlflow.pytorch.load_model(model_name)
model.eval()

In [ ]:
try:#RNN
    nb_letters = model.input_size -1
    cell_type = model.cell_type

    print("The alphabet contains", nb_letters, "symbols.")
    print("The type of the recurrent cells is", cell_type.__name__)
except: #Transformer 
    nb_letters = model.distilbert.config.vocab_size
    print("The alphabet contains", nb_letters, "symbols.")
    print("The model is a transformer (DistilBertForSequenceClassification)")

A useful function that takes the transformer and provides its output on a sequence (list of integers, as defined in Taysir): 

In [ ]:
def predict_transformer(model, word):
    """
    Note: In this function, each id in the word is added to 2 before being input to the model,
    since ids 0 and 1 are used as special tokens.
        0 : padding id
        1 : classification token id
    Args:
        word: list of integers 
    """
    word = [ [1] + [ a+2 for a in word ] ]
    word = torch.IntTensor(word)
    with torch.no_grad():
        out = model(word)
        return (out.logits.argmax().item())

In [ ]:
word = [64, 36, 48, 12, 41, 11, 9, 20, 16, 37, 23, 21, 23, 51, 52, 63, 21, 16, 28, 52, 43, 3, 3, 8, 60, 25, 23, 61, 32, 65]
predict_transformer(model, word)

## Model extraction

This is where you will extract your own model. We do not provide a baseline for transformer of Track 1.

# Submission
Save surrogatemodel as a MLFlow Model. This is the creation of the model needed for the submission to the competition. 

The only thing to do is to define a function that takes a sequence as a list of integers and returns the value given to this sequence to the sequence. Your model is **NOT** a parameter of this function. You should NOT take care of MLFlow saving here  

In [ ]:
def predict(seq):
    return predict_transformer(model, seq)

## Save and submit 
This is the creation of the model needed for the submission to the competition: you just have to run this cell. It will create in your current directory an **archive**  that you can then submit on the competition website.

**You should NOT modify this part, just run it**

In [ ]:
from submit_tools import save_function

save_function(predict, alphabet_size=nb_letters, prefix=f'dataset_{TRACK}.{DATASET}_')